In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/abigailasper/.wdm/drivers/chromedriver/mac64/102.0.5005.61]
/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_53914/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## log in to GoodReads

In [3]:
driver.get("https://www.goodreads.com/user/sign_in")

In [4]:
#select "sign in with email" option
#https://www.geeksforgeeks.org/click-button-by-text-using-python-and-selenium/
button = driver.find_element_by_link_text("Sign in with email")
button.click()

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_53914/342847067.py:3: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  button = driver.find_element_by_link_text("Sign in with email")


In [5]:
username = driver.find_element(By.NAME, "email")
username.send_keys("abigailasper@gmail.com")

### import json & keys file to safely import GoodReads password

In [6]:
import json

In [7]:
with open('../keys/gr_pw.json') as fi:
    credentials = json.load(fi)

In [8]:
gr_pw = credentials['gr_pw']

In [9]:
password = driver.find_element(By.NAME, "password")
password.send_keys(gr_pw)

In [10]:
#submit login credentials
signin = driver.find_element(By.ID, "signInSubmit")
signin.click()

## navigate to desired shelf (Booktok Books) & begin scrape

In [11]:
driver.get("https://www.goodreads.com/shelf/show/booktok")

In [16]:
#https://www.geeksforgeeks.org/how-to-scrape-multiple-pages-using-selenium-in-python/
book_list = []

for page in range(1, 26, 1):
    
    page_url = "https://www.goodreads.com/shelf/show/booktok?page=" + str(page)
    driver.get(page_url)
    title = driver.find_elements(By.CLASS_NAME, "bookTitle")
    author = driver.find_elements(By.CLASS_NAME, "authorName")
    
    for i in range(len(title)):
        book_list.append([title[i].text, author[i].text])
  
print(book_list)
        

[['The Seven Husbands of Evelyn Hugo (Hardcover)', 'Taylor Jenkins Reid'], ['It Ends with Us (Kindle Edition)', 'Colleen Hoover'], ['The Song of Achilles (Paperback)', 'Madeline Miller'], ['The Invisible Life of Addie LaRue (Hardcover)', 'V.E. Schwab'], ['The Cruel Prince (The Folk of the Air, #1)', 'Holly Black'], ['From Blood and Ash (Blood and Ash, #1)', 'Jennifer L. Armentrout'], ['Six of Crows (Six of Crows, #1)', 'Leigh Bardugo'], ['They Both Die at the End (Hardcover)', 'Adam Silvera'], ['A Court of Thorns and Roses (A Court of Thorns and Roses, #1)', 'Sarah J. Maas'], ['Red, White & Royal Blue (Paperback)', 'Casey McQuiston'], ['The Love Hypothesis (Paperback)', 'Ali Hazelwood'], ['Ugly Love (Kindle Edition)', 'Colleen Hoover'], ['We Were Liars (Kindle Edition)', 'E. Lockhart'], ['People We Meet on Vacation (Paperback)', 'Emily Henry'], ['The Spanish Love Deception (ebook)', 'Elena Armas'], ['The Hating Game (ebook)', 'Sally Thorne'], ['Beach Read (Paperback)', 'Emily Henry'], 

In [17]:
#make list a dataframe
books_df = pd.DataFrame(book_list)

books_df.head()

,0,1
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid
1,It Ends with Us (Kindle Edition),Colleen Hoover
2,The Song of Achilles (Paperback),Madeline Miller
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black


In [13]:
#get urls

urls = []

for page in range(1, 26, 1):
    
    page_url = "https://www.goodreads.com/shelf/show/booktok?page=" + str(page)
    driver.get(page_url)
    
    #https://stackoverflow.com/questions/29078072/how-to-get-link-value-inside-href
    #https://selenium-python.readthedocs.io/locating-elements.html
    hrefs = driver.find_elements(By.XPATH, "//a[@class='bookTitle']")
  
    for my_href in hrefs:
        urls.append(my_href.get_attribute("href"))
        
urls

['https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo',
 'https://www.goodreads.com/book/show/27362503-it-ends-with-us',
 'https://www.goodreads.com/book/show/13623848-the-song-of-achilles',
 'https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue',
 'https://www.goodreads.com/book/show/26032825-the-cruel-prince',
 'https://www.goodreads.com/book/show/52831200-from-blood-and-ash',
 'https://www.goodreads.com/book/show/23437156-six-of-crows',
 'https://www.goodreads.com/book/show/33385229-they-both-die-at-the-end',
 'https://www.goodreads.com/book/show/50659467-a-court-of-thorns-and-roses',
 'https://www.goodreads.com/book/show/41150487-red-white-royal-blue',
 'https://www.goodreads.com/book/show/56732449-the-love-hypothesis',
 'https://www.goodreads.com/book/show/17788401-ugly-love',
 'https://www.goodreads.com/book/show/16143347-we-were-liars',
 'https://www.goodreads.com/book/show/54985743-people-we-meet-on-vacation',
 'https://www.g

In [18]:
#add urls to dataframe
books_df['urls'] = urls
books_df.head()

,0,1,urls
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-t...
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-i...
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-t...
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-t...
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-t...


In [19]:
#checking to make sure final book id matches final book - it does!
books_df.tail()

,0,1,urls
1245,Sinners Anonymous (Sinners Anonymous #1),Somme Sketcher,https://www.goodreads.com/book/show/59818980-s...
1246,Cruel Princes (Princes of Devil's Creek Book 1),Jillian Frost,https://www.goodreads.com/book/show/60818779-c...
1247,Something Wilder (Hardcover),Christina Lauren,https://www.goodreads.com/book/show/59345253-s...
1248,Tricky Magic (Tricky Magic #1),L.L. Campbell,https://www.goodreads.com/book/show/59586927-t...
1249,"Brutal Heir (Court University, #1)",Eden O'Neill,https://www.goodreads.com/book/show/53354209-b...


In [21]:
#rename columns
books_df.columns = ['title', 'author', 'url']
books_df.head()

,title,author,url
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-t...
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-i...
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-t...
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-t...
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-t...


In [25]:
#add column that ranks books by row, as books are currently in order of # times shelved as "booktok" (most often to. least often)

books_df['booktok_popularity'] = np.arange(len(books_df))
books_df.head()

,title,author,url,booktok_popularity
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,https://www.goodreads.com/book/show/32620332-t...,0
1,It Ends with Us (Kindle Edition),Colleen Hoover,https://www.goodreads.com/book/show/27362503-i...,1
2,The Song of Achilles (Paperback),Madeline Miller,https://www.goodreads.com/book/show/13623848-t...,2
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab,https://www.goodreads.com/book/show/50623864-t...,3
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,https://www.goodreads.com/book/show/26032825-t...,4


In [27]:
#save dataframe as csv
books_df.to_csv('../data/booktok_books.csv', index=False)